# Visualize HDF5 in a knowledge graph

Using `kglab` (https://derwen.ai/docs/kgl/)

In [1]:
#!pip install kglab

In [2]:
import kglab
import rdflib

In [3]:
import h5rdmtoolbox as h5tbx

In [4]:
with h5tbx.File() as h5:
    h5.create_dataset(name='test', data=4.3)
    grp = h5.create_group(name='contact')
    grp.attrs['fname', rdflib.FOAF.firstName] = 'Matthias'
    grp.attrs['lname', rdflib.FOAF.lastName] = 'Probst'
    grp.attrs['hint'] = 'bah blah'
    grp.rdf['hint'].definition = 'A hint'
    grp.rdf.type = rdflib.PROV.Person  # --> rdf.type = Person
    grp.rdf.subject = 'https://orcid.org/0000-0001-8729-0482'  # -> @id='https://orcid.org/0000-0001-8729-0482'

    print(h5.dump_jsonld(indent=2, structural=True, resolve_keys=True))
    
g, ctx = h5tbx.jsonld.get_rdflib_graph(h5.hdf_filename)

{
  "@context": {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "hdf5": "http://purl.allotrope.org/ontologies/hdf5/1.8#",
    "prov": "http://www.w3.org/ns/prov#",
    "skos": "http://www.w3.org/2004/02/skos/core#"
  },
  "@graph": [
    {
      "@id": "_:N1",
      "@type": "hdf5:File",
      "hdf5:rootGroup": {
        "@id": "_:N0",
        "@type": "hdf5:Group",
        "hdf5:member": [
          {
            "@id": "https://orcid.org/0000-0001-8729-0482",
            "@type": [
              "hdf5:Group",
              "prov:Person"
            ],
            "foaf:firstName": "Matthias",
            "foaf:lastName": "Probst",
            "hdf5:attribute": [
              {
                "@id": "_:N2",
                "@type": "hdf5:Attribute",
                "hdf5:name": "fname",
                "hdf5:value": "Matthias"
              },
              {
                "@id": "_:N3",
                "@type": "hdf5:Attribute",
                "hdf5:name": "hint",
              

In [5]:
ctx

{'hdf5': 'http://purl.allotrope.org/ontologies/hdf5/1.8#',
 'prov': 'http://www.w3.org/ns/prov#',
 'foaf': 'http://xmlns.com/foaf/0.1/',
 'skos': 'http://www.w3.org/2004/02/skos/core#'}

In [6]:
# g = rdflib.Graph()
# bn = rdflib.BNode()
# g.add((bn, rdflib.URIRef('https://schema.org/name'), rdflib.Literal('Matthias')))

In [7]:
kg = kglab.KnowledgeGraph(
    import_graph=g,
    namespaces=ctx)

In [8]:
subgraph = kglab.SubgraphTensor(kg)

In [9]:
VIS_STYLE= {"hdf5": {"color": "red", "size": 30},
            "_": {"color": "black", "size": 20}}

In [10]:
pyvis_graph= subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.cdn_resources = 'in_line'
pyvis_graph.show('graph.html', notebook=True)

graph.html


UnicodeEncodeError: 'charmap' codec can't encode characters in position 263607-263621: character maps to <undefined>

In [ ]:
with h5tbx.File() as h5:
    h5.create_dataset(name='test', data=4.3)
    grp = h5.create_group(name='contact')
    grp.attrs['fname', rdflib.FOAF.firstName] = 'Matthias'
    grp.attrs['lname', rdflib.FOAF.lastName] = 'Probst'
    grp.attrs['hint'] = 'bah blah'
    grp.rdf['hint'].definition = 'A hint'
    grp.rdf.subject = rdflib.PROV.Person
    grp.attrs['@id'] = 'https://orcid.org/0000-0001-8729-0482'
    print(h5.dump_jsonld(indent=2, structural=False))

g, ctx = h5tbx.jsonld.get_rdflib_graph(h5.hdf_filename)

In [ ]:
jsonld = h5tbx.dump_jsonld(h5.hdf_filename, indent=2, semantic=True, structural=False)
with open('test.jsonld', 'w') as f:
    f.write(jsonld)

In [ ]:
VIS_STYLE= {"hdf5": {"color": "red", "size": 30},
            "_": {"color": "black", "size": 20},
            "prof": {"color": "green", "size": 10},
            "foaf": {"color": "green", "size": 10},
            
           }

In [ ]:
kg_from_jsonld = kglab.KnowledgeGraph().load_jsonld('test.jsonld')

subgraph = kglab.SubgraphTensor(kg_from_jsonld)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)

pyvis_graph.force_atlas_2based(overlap=0)
pyvis_graph.show("tmp.grpah.html", local=True, notebook=True)

# subgraph = kglab.SubgraphTensor(kg)
# pyvis_graph= subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
# pyvis_graph.show('graph.html')